In [1]:
from transformers import * 
import tensorflow as tf 
from tensorflow.keras.layers import * 
from tensorflow.keras.models import * 
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.text import *  
from tensorflow.keras.losses import * 
from tensorflow.keras.optimizers import *
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn
from official import nlp
import official.nlp.optimization 
import official.nlp.modeling.losses 
import official.nlp.modeling.models 
import official.nlp.modeling.networks

import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import time
import datetime

PyTorch version 1.4.0 available.
TensorFlow version 2.5.0 available.
/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


# Preprocess Data

In [2]:
train = pd.read_csv("open/train.csv") 
test = pd.read_csv("open/test.csv") 

train.shape, test.shape, train['label'].nunique()

((174304, 13), (43576, 12), 46)

In [3]:
num_classes = train['label'].nunique() 

In [4]:
train = train[['과제명','요약문_연구내용','label']] 
test = test[['과제명','요약문_연구내용']] 
train['요약문_연구내용'].fillna('NAN',inplace=True) 
test['요약문_연구내용'].fillna('NAN',inplace=True) 

In [5]:
train['data'] = train['과제명'] + train['요약문_연구내용'] 
test['data'] = test['과제명'] + test['요약문_연구내용']  

In [6]:
BATCH_SIZE = 32
NUM_EPOCHS = 30
VALID_SPLIT = 0.2 
MAX_LEN = 200 

In [7]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 
def electra_tokenizer(sent, MAX_LEN): 
    encoded_dict = tokenizer.encode_plus(text=sent, 
                                         add_special_tokens=True,
                                         max_length=MAX_LEN, 
                                         padding='max_length', 
                                         return_attention_mask=True,
                                         truncation=True)
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] 
    token_type_id = encoded_dict['token_type_ids'] 
    return input_id, attention_mask, token_type_id 

In [8]:
train_data = train['data'].values 
train_data.shape

(174304,)

In [9]:
train_labels = train['label'].values 
train_labels.shape

(174304,)

In [10]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
token_type_ids = np.zeros((N, MAX_LEN),dtype=int) 
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_data[i]
        cur_label = train_labels[i]
        input_id, attention_mask, token_type_id = electra_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        token_type_ids[i,] = token_type_id
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass
        

100%|██████████| 174304/174304 [10:34<00:00, 274.69it/s]


In [11]:
input_ids = torch.tensor(input_ids, dtype=int)

In [12]:
attention_masks = torch.tensor(attention_masks, dtype=int)

In [13]:
token_type_ids = torch.tensor(token_type_ids, dtype=int) 

labels = torch.tensor(labels, dtype=int)

In [14]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 42, test_size = 0.1) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 42, test_size = 0.1) 

train_token_ids, val_token_ids, _, _ = train_test_split(token_type_ids, labels, random_state = 42, test_size = 0.1) 


train_inputs.shape, train_attention_mask.shape, train_token_ids.shape

(torch.Size([156873, 200]),
 torch.Size([156873, 200]),
 torch.Size([156873, 200]))

In [15]:
train_labels.shape

torch.Size([156873])

In [16]:
batch_size = 32 
train_data = TensorDataset(train_inputs, train_attention_mask, train_token_ids, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_token_ids, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [17]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=46)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [18]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 30 

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps) 

In [19]:
def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [20]:
device = torch.device("cuda")

In [21]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 10 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    torch.save(model.state_dict(), "ELECTRA_BASE_" + str(epoch_i + 1)) 
print("")
print("Training complete!")



======== Epoch 1 / 30 ========
Training...
  Batch    10  of  4,903.    Elapsed: 0:00:04.
  current average loss = 3.556488347053528
  Batch    20  of  4,903.    Elapsed: 0:00:08.
  current average loss = 3.3016456604003905
  Batch    30  of  4,903.    Elapsed: 0:00:11.
  current average loss = 3.065980394681295
  Batch    40  of  4,903.    Elapsed: 0:00:15.
  current average loss = 2.828595942258835
  Batch    50  of  4,903.    Elapsed: 0:00:19.
  current average loss = 2.6077828717231752
  Batch    60  of  4,903.    Elapsed: 0:00:23.
  current average loss = 2.416169935464859
  Batch    70  of  4,903.    Elapsed: 0:00:27.
  current average loss = 2.27377770117351
  Batch    80  of  4,903.    Elapsed: 0:00:31.
  current average loss = 2.1448124930262567
  Batch    90  of  4,903.    Elapsed: 0:00:34.
  current average loss = 2.048632995949851
  Batch   100  of  4,903.    Elapsed: 0:00:38.
  current average loss = 1.9794469422101975
  Batch   110  of  4,903.    Elapsed: 0:00:42.
  curr

  Batch   910  of  4,903.    Elapsed: 0:05:50.
  current average loss = 1.0520694830260433
  Batch   920  of  4,903.    Elapsed: 0:05:54.
  current average loss = 1.0505930496298748
  Batch   930  of  4,903.    Elapsed: 0:05:58.
  current average loss = 1.0479317825968548
  Batch   940  of  4,903.    Elapsed: 0:06:02.
  current average loss = 1.045115565936616
  Batch   950  of  4,903.    Elapsed: 0:06:06.
  current average loss = 1.0423159045294712
  Batch   960  of  4,903.    Elapsed: 0:06:10.
  current average loss = 1.0400980664417148
  Batch   970  of  4,903.    Elapsed: 0:06:14.
  current average loss = 1.0377840308798958
  Batch   980  of  4,903.    Elapsed: 0:06:17.
  current average loss = 1.0366460779491735
  Batch   990  of  4,903.    Elapsed: 0:06:21.
  current average loss = 1.0340387868760812
  Batch 1,000  of  4,903.    Elapsed: 0:06:25.
  current average loss = 1.0326428476572036
  Batch 1,010  of  4,903.    Elapsed: 0:06:29.
  current average loss = 1.0304634932244179


  Batch 1,820  of  4,903.    Elapsed: 0:11:52.
  current average loss = 0.9097226284608081
  Batch 1,830  of  4,903.    Elapsed: 0:11:56.
  current average loss = 0.9084436439995557
  Batch 1,840  of  4,903.    Elapsed: 0:11:59.
  current average loss = 0.9073582141214739
  Batch 1,850  of  4,903.    Elapsed: 0:12:03.
  current average loss = 0.9064803419000393
  Batch 1,860  of  4,903.    Elapsed: 0:12:07.
  current average loss = 0.9058374977640568
  Batch 1,870  of  4,903.    Elapsed: 0:12:11.
  current average loss = 0.904396456073631
  Batch 1,880  of  4,903.    Elapsed: 0:12:15.
  current average loss = 0.9038418050855398
  Batch 1,890  of  4,903.    Elapsed: 0:12:19.
  current average loss = 0.9027308013940615
  Batch 1,900  of  4,903.    Elapsed: 0:12:23.
  current average loss = 0.9017459923891644
  Batch 1,910  of  4,903.    Elapsed: 0:12:26.
  current average loss = 0.9008563767865067
  Batch 1,920  of  4,903.    Elapsed: 0:12:30.
  current average loss = 0.8997519614795844


  Batch 2,730  of  4,903.    Elapsed: 0:17:43.
  current average loss = 0.8296293783428031
  Batch 2,740  of  4,903.    Elapsed: 0:17:47.
  current average loss = 0.828981101714129
  Batch 2,750  of  4,903.    Elapsed: 0:17:51.
  current average loss = 0.8286772439534014
  Batch 2,760  of  4,903.    Elapsed: 0:17:55.
  current average loss = 0.8277692378593096
  Batch 2,770  of  4,903.    Elapsed: 0:17:59.
  current average loss = 0.8273025164266355
  Batch 2,780  of  4,903.    Elapsed: 0:18:02.
  current average loss = 0.8265045027885077
  Batch 2,790  of  4,903.    Elapsed: 0:18:06.
  current average loss = 0.8262841525310683
  Batch 2,800  of  4,903.    Elapsed: 0:18:10.
  current average loss = 0.8252172072125332
  Batch 2,810  of  4,903.    Elapsed: 0:18:14.
  current average loss = 0.8241132051654133
  Batch 2,820  of  4,903.    Elapsed: 0:18:18.
  current average loss = 0.8235698918025967
  Batch 2,830  of  4,903.    Elapsed: 0:18:22.
  current average loss = 0.8227563444432858


  Batch 3,640  of  4,903.    Elapsed: 0:23:34.
  current average loss = 0.7676229448804816
  Batch 3,650  of  4,903.    Elapsed: 0:23:38.
  current average loss = 0.767053518454506
  Batch 3,660  of  4,903.    Elapsed: 0:23:42.
  current average loss = 0.7663069633797543
  Batch 3,670  of  4,903.    Elapsed: 0:23:46.
  current average loss = 0.7655936889939152
  Batch 3,680  of  4,903.    Elapsed: 0:23:50.
  current average loss = 0.7649309322683383
  Batch 3,690  of  4,903.    Elapsed: 0:23:53.
  current average loss = 0.7642358777484274
  Batch 3,700  of  4,903.    Elapsed: 0:23:57.
  current average loss = 0.7636700637598296
  Batch 3,710  of  4,903.    Elapsed: 0:24:01.
  current average loss = 0.7633954890172758
  Batch 3,720  of  4,903.    Elapsed: 0:24:05.
  current average loss = 0.7630253029606675
  Batch 3,730  of  4,903.    Elapsed: 0:24:09.
  current average loss = 0.7624186309548869
  Batch 3,740  of  4,903.    Elapsed: 0:24:13.
  current average loss = 0.7620215131358348


...

  Batch   840  of  4,903.    Elapsed: 0:05:24.
  current average loss = 0.013064603024118004
  Batch   850  of  4,903.    Elapsed: 0:05:28.
  current average loss = 0.013097436581464375
  Batch   860  of  4,903.    Elapsed: 0:05:32.
  current average loss = 0.013033776448736357
  Batch   870  of  4,903.    Elapsed: 0:05:36.
  current average loss = 0.0129671027905297
  Batch   880  of  4,903.    Elapsed: 0:05:39.
  current average loss = 0.012888623092492873
  Batch   890  of  4,903.    Elapsed: 0:05:43.
  current average loss = 0.012816975939642178
  Batch   900  of  4,903.    Elapsed: 0:05:47.
  current average loss = 0.012799786838392417
  Batch   910  of  4,903.    Elapsed: 0:05:51.
  current average loss = 0.012694129753080043
  Batch   920  of  4,903.    Elapsed: 0:05:55.
  current average loss = 0.012683896667769421
  Batch   930  of  4,903.    Elapsed: 0:05:59.
  current average loss = 0.012650392577052116
  Batch   940  of  4,903.    Elapsed: 0:06:03.
  current average loss = 

  Batch 1,730  of  4,903.    Elapsed: 0:11:09.
  current average loss = 0.012279938680329764
  Batch 1,740  of  4,903.    Elapsed: 0:11:13.
  current average loss = 0.012255096994340419
  Batch 1,750  of  4,903.    Elapsed: 0:11:17.
  current average loss = 0.012257418213146073
  Batch 1,760  of  4,903.    Elapsed: 0:11:21.
  current average loss = 0.012331285334022885
  Batch 1,770  of  4,903.    Elapsed: 0:11:25.
  current average loss = 0.012351671440032243
  Batch 1,780  of  4,903.    Elapsed: 0:11:28.
  current average loss = 0.01232189011833306
  Batch 1,790  of  4,903.    Elapsed: 0:11:32.
  current average loss = 0.01225440806641592
  Batch 1,800  of  4,903.    Elapsed: 0:11:36.
  current average loss = 0.012198582198470831
  Batch 1,810  of  4,903.    Elapsed: 0:11:40.
  current average loss = 0.012161988972876611
  Batch 1,820  of  4,903.    Elapsed: 0:11:44.
  current average loss = 0.01216334100239552
  Batch 1,830  of  4,903.    Elapsed: 0:11:48.
  current average loss = 0

  Batch 2,620  of  4,903.    Elapsed: 0:16:59.
  current average loss = 0.012583828079052098
  Batch 2,630  of  4,903.    Elapsed: 0:17:03.
  current average loss = 0.012542453163193659
  Batch 2,640  of  4,903.    Elapsed: 0:17:07.
  current average loss = 0.012516683378172192
  Batch 2,650  of  4,903.    Elapsed: 0:17:11.
  current average loss = 0.012523086423862656
  Batch 2,660  of  4,903.    Elapsed: 0:17:15.
  current average loss = 0.01251694226035274
  Batch 2,670  of  4,903.    Elapsed: 0:17:18.
  current average loss = 0.01248820290555445
  Batch 2,680  of  4,903.    Elapsed: 0:17:22.
  current average loss = 0.01246850033944
  Batch 2,690  of  4,903.    Elapsed: 0:17:26.
  current average loss = 0.012471161958619564
  Batch 2,700  of  4,903.    Elapsed: 0:17:30.
  current average loss = 0.012482461583007264
  Batch 2,710  of  4,903.    Elapsed: 0:17:34.
  current average loss = 0.012556952371423534
  Batch 2,720  of  4,903.    Elapsed: 0:17:38.
  current average loss = 0.01

  Batch 3,510  of  4,903.    Elapsed: 0:22:47.
  current average loss = 0.013166307141086315
  Batch 3,520  of  4,903.    Elapsed: 0:22:50.
  current average loss = 0.01317753233756362
  Batch 3,530  of  4,903.    Elapsed: 0:22:54.
  current average loss = 0.013183514321112092
  Batch 3,540  of  4,903.    Elapsed: 0:22:58.
  current average loss = 0.013184344944265266
  Batch 3,550  of  4,903.    Elapsed: 0:23:02.
  current average loss = 0.013265116885304452
  Batch 3,560  of  4,903.    Elapsed: 0:23:06.
  current average loss = 0.013267139704332928
  Batch 3,570  of  4,903.    Elapsed: 0:23:10.
  current average loss = 0.013290764690906393
  Batch 3,580  of  4,903.    Elapsed: 0:23:14.
  current average loss = 0.013300574057886387
  Batch 3,590  of  4,903.    Elapsed: 0:23:17.
  current average loss = 0.01342765946553444
  Batch 3,600  of  4,903.    Elapsed: 0:23:21.
  current average loss = 0.013408442829839058
  Batch 3,610  of  4,903.    Elapsed: 0:23:25.
  current average loss = 

  Batch 4,400  of  4,903.    Elapsed: 0:28:31.
  current average loss = 0.013853477074510671
  Batch 4,410  of  4,903.    Elapsed: 0:28:35.
  current average loss = 0.013925284279886287
  Batch 4,420  of  4,903.    Elapsed: 0:28:38.
  current average loss = 0.013931859599013404
  Batch 4,430  of  4,903.    Elapsed: 0:28:42.
  current average loss = 0.013916193067388394
  Batch 4,440  of  4,903.    Elapsed: 0:28:46.
  current average loss = 0.013889745237758835
  Batch 4,450  of  4,903.    Elapsed: 0:28:50.
  current average loss = 0.01387702044225141
  Batch 4,460  of  4,903.    Elapsed: 0:28:54.
  current average loss = 0.013904631359193625
  Batch 4,470  of  4,903.    Elapsed: 0:28:58.
  current average loss = 0.013893470742325121
  Batch 4,480  of  4,903.    Elapsed: 0:29:01.
  current average loss = 0.013946710707386956
  Batch 4,490  of  4,903.    Elapsed: 0:29:05.
  current average loss = 0.01396089267807044
  Batch 4,500  of  4,903.    Elapsed: 0:29:09.
  current average loss = 

# Make Prediction

Let us use the model from epoch 30. It seems to have the best training loss as well as best validation accuracy. 

In [24]:
test_data = test['data'].values 

In [26]:
## create test set batches 
N = test_data.shape[0] 

test_input_ids = np.zeros((N, MAX_LEN),dtype=int)
test_attention_masks = np.zeros((N, MAX_LEN),dtype=int)
test_token_type_ids = np.zeros((N, MAX_LEN),dtype=int) 

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = test_data[i]
        input_id, attention_mask, token_type_id = electra_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        test_input_ids[i,] = input_id 
        test_attention_masks[i,] = attention_mask 
        test_token_type_ids[i,] = token_type_id
    except Exception as e: 
        print(e)
        print(cur_str)
        pass
        

100%|██████████| 43576/43576 [02:53<00:00, 251.84it/s]


In [27]:
test_input_ids = torch.tensor(test_input_ids, dtype=int) 
test_attention_masks = torch.tensor(test_attention_masks, dtype=int) 
test_token_type_ids = torch.tensor(test_token_type_ids, dtype=int) 

In [28]:
batch_size = 32 
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_token_type_ids) 
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [29]:
checkpoint = torch.load('ELECTRA_BASE_30') 
test_model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=46)
test_model.load_state_dict(checkpoint) 
test_model.cuda() 

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [38]:
predictions = []

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
test_model.eval()


# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_token_type_ids = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask)
    # 로짓 구함 
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()
    predictions.append(pred_flat)

    
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,362.    Elapsed: 0:00:12.
  Batch   200  of  1,362.    Elapsed: 0:00:23.
  Batch   300  of  1,362.    Elapsed: 0:00:35.
  Batch   400  of  1,362.    Elapsed: 0:00:46.
  Batch   500  of  1,362.    Elapsed: 0:00:58.
  Batch   600  of  1,362.    Elapsed: 0:01:09.
  Batch   700  of  1,362.    Elapsed: 0:01:21.
  Batch   800  of  1,362.    Elapsed: 0:01:33.
  Batch   900  of  1,362.    Elapsed: 0:01:44.
  Batch 1,000  of  1,362.    Elapsed: 0:01:56.
  Batch 1,100  of  1,362.    Elapsed: 0:02:08.
  Batch 1,200  of  1,362.    Elapsed: 0:02:19.
  Batch 1,300  of  1,362.    Elapsed: 0:02:31.
Test took: 0:02:38


In [42]:
pred_flattened = [] 
for pred in predictions: 
    for j in range(len(pred)): 
        pred_flattened.append(pred[j]) 

In [32]:
submission = pd.read_csv('open/sample_submission.csv') 
submission.head()  

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0


In [45]:
submission.iloc[:,1] = pred_flattened

In [47]:
submission.to_csv("ELECTRA_BASELINE_SUBMISSION.csv",index=False)